In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
averages_multi_queue = pd.read_csv('results_directory/same_patient_profiles/multi_queue/current_state/averages_data.csv')
averages_single_queue = pd.read_csv('results_directory/same_patient_profiles/single_queue/current_state/averages_data.csv')

# calculate the mean of the averages
mean_multi_queue = averages_multi_queue.mean()
mean_single_queue = averages_single_queue.mean()

### Here we compare both simulations which were ran with identical patient profiles

### Essentially, we have a sample of pairs $$(X_1, Y_1), (X_2, Y_2),\dots, (X_{100}, Y_{100})$$
### We can now create a sample of differences $$D_i = (X_1 - Y_i)$$
### Now $D_1, D_2, \dots D_{100}$ are i.i.d. 
### Additionally, $$\mathbf{E}(D_i) = \mathbf{E}(X_i) - \mathbf{E}(Y_i)$$
### Our only assumption from here on out will be that $D_i\sim\mathcal{N}(\mathbf{E}(D_i), \mathbf{Var}(D_i))$
### This is not a groundless assumption, since, for sufficiently large $n$, $X$ and $Y$, which are averages, should be approximatelly normally distributed. as such, their difference, as a linear function of two normally distributed variables, would also be normally distributed.

#### Credit to Technion statistics 1 course - presentation 10

In [3]:
# I want to create two dfs like the ones loaded but without columns containing the word "nurse"
nurse_columns_multi_queue = [col for col in averages_multi_queue.columns if "nurse" in col]
nurse_columns_single_queue = [col for col in averages_single_queue.columns if "nurse" in col]

no_nurse_multi_queue = averages_multi_queue.drop(columns=nurse_columns_multi_queue)
no_nurse_single_queue = averages_single_queue.drop(columns=nurse_columns_single_queue)

In [4]:
differences_df = no_nurse_multi_queue - no_nurse_single_queue

In [10]:
differences_df.columns

Index(['Unnamed: 0', 'q_flow_station_wait_time_avg',
       'q_flow_station_queue_length_avg', 'q_flow_station_busy_avg',
       'secretary_station_wait_time_avg', 'secretary_station_queue_length_avg',
       'secretary_station_busy_avg', 'leukemia_doctor_1_wait_time_avg',
       'leukemia_doctor_1_queue_length_avg', 'leukemia_doctor_1_busy_avg',
       'leukemia_doctor_2_wait_time_avg', 'leukemia_doctor_2_queue_length_avg',
       'leukemia_doctor_2_busy_avg', 'transplant_doctor_1_wait_time_avg',
       'transplant_doctor_1_queue_length_avg', 'transplant_doctor_1_busy_avg',
       'transplant_doctor_2_wait_time_avg',
       'transplant_doctor_2_queue_length_avg', 'transplant_doctor_2_busy_avg',
       'transplant_doctor_3_wait_time_avg',
       'transplant_doctor_3_queue_length_avg', 'transplant_doctor_3_busy_avg',
       'other_patients_total_processing_time_avg',
       'leukemia_doctor_1_complex_patients_total_processing_time_avg',
       'leukemia_doctor_1_regular_patients_total_p

### Our hypothesis $H_0$ is - $\mu_d = d_0$.
### In our case $n = 100$; $d_0 = 0$

### We make two observations:
### - Under $H_0$: $T = \frac{\bar{D}-d_0}{\frac{S_D}{\sqrt{n}}}\sim t_{(n-1)}$
### - $S_D^{2} = \frac{1}{n-1} \sum_{i=1}^{n} \left(D_i - \bar{D}\right)^2$

### Thus, a confidence interval at confidence level $1 - \alpha$ is:
### $$\left[\bar{D} \pm t_{(n-1),1-\frac{\alpha}{2}}\frac{S_D}{\sqrt{n}}\right]$$

#### Credit to Technion statistics 1 course - presentation 10

In [11]:
def S_D(df, column_name):
    """
    Calculate the sample standard deviation for a given column in a DataFrame.
    """
    return np.std(df[column_name], ddof=1) # ddof=1 to ensure the sample standard deviation is an unbiased estimator of the population standard deviation

def D_bar(df, column_name):
    """
    Calculate the sample mean for a given column in a DataFrame.
    """
    return np.mean(df[column_name])

def calculate_confidence_interval(df, column_name, confidence_level=0.95):
    """
    Calculate the confidence interval for a given column in a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The name of the column to calculate the confidence interval for.
        confidence_level (float): The confidence level for the interval (default is 0.95).
        
    Returns:
        tuple: A tuple containing the lower and upper bounds of the confidence interval.
    """

    n = len(df)
    t_value = stats.t.ppf(1 - (1 - confidence_level) / 2, n - 1)
    lower_bound = D_bar(df, column_name) - t_value * S_D(df, column_name) / np.sqrt(n)
    upper_bound = D_bar(df, column_name) + t_value * S_D(df, column_name) / np.sqrt(n)
    return lower_bound, upper_bound

def does_null_hypothesis_hold(df, column_name, confidence_level=0.95):
    """
    Check if the null hypothesis holds for a given column in a DataFrame.
    """
    lower_bound, upper_bound = calculate_confidence_interval(df, column_name, confidence_level)
    print(f"Column name: {column_name}")
    print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    print(f"Null hypothesis holds: {lower_bound <= 0 <= upper_bound}")
    print(f"{'#'*50}")
    return lower_bound <= 0 <= upper_bound

def does_null_hypothesis_hold_for_all_columns(df, confidence_level=0.95):
    """
    Check if the null hypothesis holds for all columns in a DataFrame.
    """
    for column in df.columns:
        if not does_null_hypothesis_hold(df, column, confidence_level):
            return False
    return True

def csv_of_results(df, column_name, confidence_level=0.95):
    # Create results dataframe with proper columns
    results_df = pd.DataFrame(columns=['lower_bound', 'upper_bound', 'null_hypothesis_holds'])
    
    for column in df.columns:
        try:
            lower_bound, upper_bound = calculate_confidence_interval(df, column, confidence_level)
            null_hypothesis_result = does_null_hypothesis_hold(df, column, confidence_level)
            
            # Add row using loc with proper indexing
            results_df.loc[column] = {
                'lower_bound': lower_bound,
                'upper_bound': upper_bound, 
                'null_hypothesis_holds': null_hypothesis_result
            }
        except Exception as e:
            print(f"Error processing column {column}: {e}")
            # Add row with NaN values if there's an error
            results_df.loc[column] = {
                'lower_bound': np.nan,
                'upper_bound': np.nan,
                'null_hypothesis_holds': np.nan
            }
    
    results_df.to_csv(f"results_directory/statistical_analysis/{column_name}.csv")
    return results_df

In [12]:
csv_of_results(differences_df, "differences_df")

Column name: Unnamed: 0
Lower bound: 0.0, Upper bound: 0.0
Null hypothesis holds: True
##################################################


ValueError: cannot set a row with mismatched columns

In [9]:
does_null_hypothesis_hold_for_all_columns(differences_df, 0.95)

Column name: Unnamed: 0
Lower bound: 0.0, Upper bound: 0.0
Null hypothesis holds: True
##################################################
Column name: q_flow_station_wait_time_avg
Lower bound: -0.044476209632849664, Upper bound: 0.04768416744189949
Null hypothesis holds: True
##################################################
Column name: q_flow_station_queue_length_avg
Lower bound: -0.006190289296514656, Upper bound: 0.004386939962445271
Null hypothesis holds: True
##################################################
Column name: q_flow_station_busy_avg
Lower bound: -0.009190910781360776, Upper bound: 0.001379415212729664
Null hypothesis holds: True
##################################################
Column name: secretary_station_wait_time_avg
Lower bound: -0.48569281542137643, Upper bound: 0.20516260291974017
Null hypothesis holds: True
##################################################
Column name: secretary_station_queue_length_avg
Lower bound: -0.06857685140439146, Upper bound: 0.01

False